<a href="https://colab.research.google.com/github/nomo82/python/blob/main/Meta_Sheets_Cali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Extracción Leads Meta (Facebook) Cali (Total)
# Instalar bibliotecas necesarias
!pip install gspread oauth2client pandas pydrive2 --quiet

# Importar bibliotecas
from oauth2client.service_account import ServiceAccountCredentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import pandas as pd
import gspread
import time

# Autenticación de Google Colab
from google.colab import auth
auth.authenticate_user()

# Subir archivo JSON de credenciales
from google.colab import files
uploaded = files.upload()

# Configuración de acceso
scope = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/spreadsheets'
]
credentials = ServiceAccountCredentials.from_json_keyfile_name(list(uploaded.keys())[0], scope)
gc = gspread.authorize(credentials)

# Autenticación de PyDrive
gauth = GoogleAuth()
gauth.credentials = credentials
drive = GoogleDrive(gauth)

# ID de la carpeta de Google Drive (Pereira)
folder_id = '1dbMkDwKaw4g7qC35Yviymh_-2Kjfm4Ds'  # URL Carpeta pauta Meta Cali

# Obtener archivos de la carpeta
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'"}).GetList()

# Crear un DataFrame para almacenar los datos combinados
all_data = pd.DataFrame()

# Limitar las solicitudes por minuto (máximo 60)
requests_count = 0
REQUESTS_LIMIT = 60

# Procesar cada archivo
for file in file_list:
    print(f"Procesando archivo: {file['title']}")
    try:
        # Control de solicitudes
        if requests_count >= REQUESTS_LIMIT:
            print("Límite de solicitudes alcanzado. Esperando 60 segundos...")
            time.sleep(60)
            requests_count = 0

        # Abrir la hoja de Google Sheets
        sheet = gc.open_by_key(file['id']).sheet1
        requests_count += 1  # Contar solicitud

        # Obtener los encabezados y todos los datos
        data = sheet.get_all_values()
        headers = data[0]  # Primera fila como encabezados
        rows = data[1:]    # Excluir encabezados

        # Crear un DataFrame con los datos
        df = pd.DataFrame(rows, columns=headers)

        # Agregar metadatos del archivo
        df['Nombre de archivo'] = file['title']

        # Concatenar los datos al DataFrame principal
        all_data = pd.concat([all_data, df], ignore_index=True)

    except Exception as e:
        print(f"Error procesando {file['title']}: {e}")
        continue

# Verificar y exportar los datos combinados con codificación 'utf-8'
if not all_data.empty:
    output_file = 'datos_combinados_cali_facebook.csv'
    all_data.to_csv(output_file, index=False, sep=';', encoding='utf-8')
    print(f'Datos exportados a {output_file} con separador ";" y codificación UTF-8')
else:
    print("No se encontraron datos para exportar.")

Saving python-442121-1e532c9dedf4.json to python-442121-1e532c9dedf4.json
Procesando archivo: CHIPICHAPE
Procesando archivo: Altavista 2000 
Procesando archivo: Locales Cali
Datos exportados a datos_combinados_cali_facebook.csv con separador ";" y codificación UTF-8


In [2]:
# Verificar y exportar los datos combinados con codificación 'utf-8'
if not all_data.empty:
    # Imprimir el DataFrame combinado en pantalla
    print(all_data)

    output_file = 'datos_combinados_cali_facebook.csv'
    all_data.to_csv(output_file, index=False, sep=';', encoding='utf-8')
    print(f'Datos exportados a {output_file} con separador ";" y codificación UTF-8')
else:
    print("No se encontraron datos para exportar.")

      Codigo Proyecto      Fecha registro Estado Nombres  \
0           Cal-CHIPI  5/01/2023 15:28:59    New           
1           Cal-CHIPI  5/01/2023 15:28:59    New           
2           Cal-CHIPI  5/01/2023 15:28:59    New           
3           Cal-CHIPI  5/01/2023 15:28:59    New           
4           Cal-CHIPI  5/01/2023 15:28:59    New           
...               ...                 ...    ...     ...   
38799             NaN                 NaN    NaN     NaN   
38800             NaN                 NaN    NaN     NaN   
38801             NaN                 NaN    NaN     NaN   
38802             NaN                 NaN    NaN     NaN   
38803             NaN                 NaN    NaN     NaN   

                            Apellidos       Celular  \
0                          paula cruz    3113824367   
1                            Carolina  573175763882   
2                         Laura Ferro  573144289123   
3          Olga Milena Guevara Claros  573153372079   
4   

In [3]:
#Determinamos las columnas que deseamos visualizar ver ejemplo:['Codigo proyecto', 'Fecha registro', 'Estado']
print(all_data[['Codigo Proyecto', 'Fecha registro', 'Estado']].head(100))

   Codigo Proyecto      Fecha registro Estado
0        Cal-CHIPI  5/01/2023 15:28:59    New
1        Cal-CHIPI  5/01/2023 15:28:59    New
2        Cal-CHIPI  5/01/2023 15:28:59    New
3        Cal-CHIPI  5/01/2023 15:28:59    New
4        Cal-CHIPI  5/01/2023 15:28:59    New
..             ...                 ...    ...
95       Cal-CHIPI  5/01/2023 15:28:59    New
96       Cal-CHIPI  5/01/2023 15:28:59    New
97       Cal-CHIPI  5/01/2023 15:28:59    New
98       Cal-CHIPI  5/01/2023 15:28:59    New
99       Cal-CHIPI  5/01/2023 15:28:59    New

[100 rows x 3 columns]


In [4]:
#Cantidad de datos del DF para la pauta de Facebook ADS Cali
all_data.index

RangeIndex(start=0, stop=38804, step=1)

In [ ]:
#Descargar archivo Cali de GoogleColab al PC
from google.colab import files
files.download('datos_combinados_cali_facebook.csv')